<a href="https://colab.research.google.com/github/MuhammadShavaiz/AI_learning/blob/main/bigram_LLM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import numpy as np

In [2]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()
print(len(text))

233427


In [4]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
82


In [6]:
char_to_int = {ch:i for i ,ch in enumerate(chars)}
int_to_char = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [char_to_int[c] for c in x]
decode = lambda y: ''.join([int_to_char[i] for i in y])
encoded_shavaiz = encode('shavaiz')
decoded_shavaiz = decode(encoded_shavaiz)
print(encoded_shavaiz, decoded_shavaiz)

[74, 63, 56, 77, 56, 64, 81] shavaiz


In [8]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69, 57,
        60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 30, 70, 73, 70, 75,
        63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,  1,
        64, 69,  1, 41, 81,  0,  1,  1,  1,  1,  0, 46, 63, 64, 74,  1, 60, 57,
        70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63, 60,  1, 76, 74, 60,
         1, 70, 61,  1, 56, 69, 80, 70, 69, 60])


In [12]:
print(decode(data[:100].numpy()))

The Project Gutenberg eBook of Dorothy and the Wizard in Oz
    
This ebook is for the use of anyone


In [13]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
block_size = 8
batch_size = 4

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f'when context is {context}, target is {target}')

when context is tensor([46]), target is 63
when context is tensor([46, 63]), target is 60
when context is tensor([46, 63, 60]), target is 1
when context is tensor([46, 63, 60,  1]), target is 42
when context is tensor([46, 63, 60,  1, 42]), target is 73
when context is tensor([46, 63, 60,  1, 42, 73]), target is 70
when context is tensor([46, 63, 60,  1, 42, 73, 70]), target is 65
when context is tensor([46, 63, 60,  1, 42, 73, 70, 65]), target is 60
